In [9]:
import requests
import time
#!pip install scikit-learn llama-cpp-python

def query_mistral(model, prompt):
    url = "http://ollama:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
    }

    start_time = time.time()
    response = requests.post(url, json=payload)
    end_time = time.time()

    result = response.json()["response"]
    duration = end_time - start_time

    print(f"Response: {result}")
    print(f"\n⏱️ Time taken: {duration:.2f} seconds")

query_mistral("mistral", "Was ist die Gemeinsamkeit von Julius und Sven?")
#query_mistral("llama2", "Kannst du mir auf Deutsch antworten?")

Response:  Julius und Sven sind beide männliche Vornamen, was bedeutet, dass sie gleiche Personennamen sind.

⏱️ Time taken: 37.06 seconds


In [4]:
import os

folder_path = "test_data"
text_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

documents = {}
for file_name in text_files:
    with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
        documents[file_name] = file.read()
all_text = "\n\n".join(documents.values())
documents

{'Handbuch.txt': 'Die Sahnemaschine muss täglich und wöchentlich gereinigt werden.\n \nTägliche Desinfektionsreinigung:\n \nSaugrohr und den Sahnetopf herausnehmen und unter fließendem Wasser reinigen\nSpültopf mit 1,5 L warmem Wasser füllen und in den Sanomat setzen\nEimer unter die Düse stellen\nReinigungstaste drücken\nTauschen der Garniertülle gegen die Reinigungsdüse\nSpültopf mit 1,5 L  warmem Wasser und 30 ml SANO-Desinfektion füllen\nReinigungstaste drücken\nSpültopf reinigen\nZur Nachreinigung 1,5 L kaltes Wasser in den Spültopf füllen\nReinigungstaste drücken\nReinigungsdüse durch die gereinigte Garniertülle ersetzen\nEin Tuch mit SANO-Desinfektion tränken und die Maschine von außen und innen reinigen\nDas Gerät über Nacht eingeschaltet lassen\n \n \nWöchentliche Desinfektionsreinigung:\n \nSaugrohr aus dem Gehäuse ziehen\nGehäuse vom Ventilkopf abziehen\nLuftregulierungsschraube abschrauben\nGabel herausziehen\nGabel von unten in das Gehäuse stoßen und die Luftdüsenscheibe u

In [27]:
question = "Was ist, wenn ich mehr als 1,5 Liter in den Spültopf gemacht habe?"
full_prompt = f"""Bitte folgende Frage beantworten: "{question}". 
Der Input zur Beantwortung der Frage wie folgt. :
{all_text}
"""
query_mistral("mistral", full_prompt)

Response:  Wenn du mehr als 1,5 Liter in den Spültopf gemacht hast, mache folgendes um die Menge korrigieren zu können:

   Wenn du mehr als 1,5 Liter Wasser im Spültopf hast, dann solltest du das Wasser aus dem Spültopf entleeren und es bis auf das richtige Maß von 1,5 Litern füllen. Dann kannst du mit der Täglichen Desinfektionsreinigung fortfahren, indem du den Tap in den Spültopf legst und die Verfahren wie oben beschrieben ausführst.

⏱️ Time taken: 97.67 seconds


In [6]:
# Load and chunk document
def chunk_text(text, chunk_size=300, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        chunks.append(text[start:start+chunk_size])
        start += chunk_size - overlap
    return chunks

docs = []
paths = [os.path.join("test_data", f) for f in os.listdir("test_data") if f.endswith(".txt")]
for path in paths:
    with open(path, "r", encoding="utf-8") as f:
        chunks = chunk_text(f.read())
        docs.extend(chunks)
docs

['Die Sahnemaschine muss täglich und wöchentlich gereinigt werden.\n \nTägliche Desinfektionsreinigung:\n \nSaugrohr und den Sahnetopf herausnehmen und unter fließendem Wasser reinigen\nSpültopf mit 1,5 L warmem Wasser füllen und in den Sanomat setzen\nEimer unter die Düse stellen\nReinigungstaste drücken\nTa',
 ' unter die Düse stellen\nReinigungstaste drücken\nTauschen der Garniertülle gegen die Reinigungsdüse\nSpültopf mit 1,5 L  warmem Wasser und 30 ml SANO-Desinfektion füllen\nReinigungstaste drücken\nSpültopf reinigen\nZur Nachreinigung 1,5 L kaltes Wasser in den Spültopf füllen\nReinigungstaste drücken\nReini',
 ' den Spültopf füllen\nReinigungstaste drücken\nReinigungsdüse durch die gereinigte Garniertülle ersetzen\nEin Tuch mit SANO-Desinfektion tränken und die Maschine von außen und innen reinigen\nDas Gerät über Nacht eingeschaltet lassen\n \n \nWöchentliche Desinfektionsreinigung:\n \nSaugrohr aus dem Gehäuse zi',
 'infektionsreinigung:\n \nSaugrohr aus dem Gehäuse ziehen\n

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorize chunks
vectorizer = TfidfVectorizer().fit(docs)
doc_vectors = vectorizer.transform(docs)

In [10]:
# RAG query
def rag_query(question, top_k=3):
    q_vec = vectorizer.transform([question])
    scores = cosine_similarity(q_vec, doc_vectors)[0]
    top_idxs = scores.argsort()[-top_k:][::-1]
    
    context = "\n\n".join(docs[i] for i in top_idxs)
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"

    query_mistral("mistral", prompt)

question = "Wann muss ich den Ventilkopf abziehen?"
rag_query(question)

Response:  Du musst den Ventilkopf abziehen während der wöchentlichen infektionsreinigung.

⏱️ Time taken: 67.75 seconds
